<a href="https://colab.research.google.com/github/Minseos/KISTI_AI/blob/main/%EC%BD%94%EB%9E%A9_%EA%B3%B5%EB%AA%A8%EC%A0%844.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !pip install datasets
# !pip install transformers


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
from datasets import Dataset
from transformers import BertTokenizer

# JSON 파일 경로
json_file_path = "/content/drive/MyDrive/라벨링데이터2.json"

# JSON 파일을 KcBERT 학습용 텍스트 데이터로 변환
def convert_json_to_dataset(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    qa_pairs = []
    for item in data:
        question = item.get("question")
        answer = item.get("answer")
        if question and answer:
            qa_pairs.append({"question": question, "answer": answer})

    dataset = Dataset.from_list(qa_pairs)
    return dataset

# 1. 데이터를 변환하여 Hugging Face Dataset으로 저장
qa_dataset = convert_json_to_dataset(json_file_path)

# 2. 토크나이저 로드
model_name = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)

# 3. 데이터셋 토크나이즈
def preprocess_data(examples):
    return tokenizer(examples['question'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = qa_dataset.map(preprocess_data, batched=True)

print(f"데이터셋 크기: {len(tokenized_dataset)}")


Map:   0%|          | 0/35233 [00:00<?, ? examples/s]

In [17]:
# JSON 파일이 제대로 열리는지 확인
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 파일 내 첫 번째 항목을 출력하여 구조 확인
print(f"첫 번째 항목: {data[0]}")
print(f"총 항목 수: {len(data)}")


첫 번째 항목: {'question': '우리 사장이나 간부들이 이것을 대단히 소홀하게 중요하지 않게 생각하는데 끝까지 내 이야기에 대한 답변이 나오지 않았습니다.  공역문제인데 아까 공역을 전부 예를 들었습니다마는 가령 미공군 매향리 사격장, 한ㆍ미합동훈련장 또 여주 공군사격장, 수도권 비행금지구역, 휴전선 비행금지구역 이렇게 공역이 되어 있는데 앞으로 허브공항으로서 많은 비행기가 오는 곳에 공역이 도처에 있어요. 과연 그 공역을 피해 갈 수 있을 것인가에 대한 어려움을 우리 건교부하고 미8군하고 국방부하고 협의해서 공역문제는 사전에 합의를 보아야 할 문제인데 이것에 대해서 사장은 어떤 생각을 가지고 있느냐고 물어 보았습니다. 답변을 해주십시오.', '회의번호': 30045, '질의응답번호': 1, '회의구분': '국정감사', '위원회': '건설교통위원회', '회의일자': '2000年10月19日(木)', '질문자': '3337', '질문자_ISNI': '0000000463657210', 'answer': '서면으로 답변을 드리려고 했습니다마는 공역문제는 너무 중요한 문제입니다. 그리고 당장은 아니지만 2, 3년 앞을 내다보면 매우 심각한 문제입니다. 미국 정부에서도 그 심각성을 인정하고 있습니다. 그래서 제가 알기로는 건교부에서 국방부, 미 당국과도 협의를 이미 진행하고 있습니다.   그래서 공역을 변경하기로 일부 합의되어서 저희 공항개항과 더불어 현재의 공역보다도 많이 여유를 갖게 되어 있습니다. 그리고 근본적인 문제는 앞으로 한ㆍ미간 또는 국내에서 국방부와 협의가 계속 진행될 것으로 알고 있습니다.', '답변자': ' ', '답변자_ISNI': ' '}
총 항목 수: 35233


In [18]:
import json
from datasets import Dataset

# JSON 파일을 KcBERT 학습용 텍스트 데이터로 변환
def convert_json_to_dataset(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    qa_pairs = []
    for item in data:
        question = item.get("question", "")
        answer = item.get("answer", "")

        if question and answer:  # question과 answer가 있는 경우에만 추가
            qa_pairs.append({
                "question": question,
                "answer": answer,
                # 'context' 대신 바로 question과 answer를 사용
            })

    dataset = Dataset.from_list(qa_pairs)
    return dataset

# 데이터셋 변환
qa_dataset = convert_json_to_dataset(json_file_path)
print(f"데이터셋 크기: {len(qa_dataset)}")


데이터셋 크기: 35233


토크나이즈된 데이터셋 크기: 0


데이터 토크나이즈 및 학습 진행 코드:

In [1]:
import torch
from transformers import BertTokenizerFast, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset

# 1. Fast 토크나이저 로드
model_name = "beomi/kcbert-base"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# 2. 데이터셋 전처리 (question과 answer만 사용)
def preprocess_data(examples):
    # 질문과 답변을 배치 단위로 처리 (리스트로 제공됨)
    tokenized_examples = tokenizer(
        examples["question"],
        examples["answer"],  # answer를 context 대신 사용
        truncation=True,
        padding="max_length",
        max_length=300,  # 최대 길이를 300으로 변경
        return_offsets_mapping=True  # 오프셋 매핑을 사용하여 답변 위치 계산
    )

    # 배치 단위로 각 질문/답변의 시작과 끝 위치를 계산
    start_positions = []
    end_positions = []

    for i, (context, answer) in enumerate(zip(examples["answer"], examples["answer"])):
        start_pos = context.find(answer)
        end_pos = start_pos + len(answer)

        if start_pos == -1:
            start_pos = 0
            end_pos = 0

        # 오프셋 매핑을 사용하여 실제 토큰화된 문장에서 시작과 끝 위치 찾기
        offsets = tokenized_examples["offset_mapping"][i]
        token_start_index = 0
        token_end_index = len(offsets) - 1

        for idx, offset in enumerate(offsets):
            if offset[0] <= start_pos < offset[1]:
                token_start_index = idx
            if offset[0] <= end_pos < offset[1]:
                token_end_index = idx

        start_positions.append(token_start_index)
        end_positions.append(token_end_index)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    return tokenized_examples

# 3. 데이터셋에 전처리 적용
tokenized_dataset = qa_dataset.map(preprocess_data, batched=True)

# 4. KcBERT 모델 로드 (질문-답변용)
model = BertForQuestionAnswering.from_pretrained(model_name)

# 5. 학습 하이퍼파라미터 설정
training_args = TrainingArguments(
    output_dir="./kcbert_finetuned",   # 학습된 모델 저장 위치
    overwrite_output_dir=True,         # 기존 저장된 모델 덮어쓰기
    num_train_epochs=1,                # 학습할 에폭 수
    per_device_train_batch_size=8,    # 배치 크기
    save_steps=20_000,                 # 체크포인트 저장 간격
    save_total_limit=2,                # 체크포인트 최대 저장 개수
    logging_dir="./logs",              # 로그 저장 디렉토리
    logging_steps=500,                 # 로그 기록 간격
    remove_unused_columns=False,       # 모든 열을 사용하도록 설정
    fp16=True,                         # 16비트 부동소수점 사용
)

# 6. Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=lambda data: {
        "input_ids": torch.tensor([f["input_ids"] for f in data]),
        "attention_mask": torch.tensor([f["attention_mask"] for f in data]),
        "token_type_ids": torch.tensor([f["token_type_ids"] for f in data]),
        "start_positions": torch.tensor([f["start_positions"] for f in data]),
        "end_positions": torch.tensor([f["end_positions"] for f in data]),
    },
)

# 7. 모델 학습 시작
trainer.train()

# 8. 학습 완료된 모델 저장
trainer.save_model('./kcbert_finetuned')
tokenizer.save_pretrained('./kcbert_finetuned')

print("KcBERT 모델 미세 조정 완료 및 저장되었습니다!")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


NameError: name 'qa_dataset' is not defined

1. 모델 평가 및 테스트

In [ ]:
import torch
from transformers import BertTokenizerFast, BertForQuestionAnswering

# 학습된 모델과 토크나이저 로드
model = BertForQuestionAnswering.from_pretrained('./kcbert_finetuned')
tokenizer = BertTokenizerFast.from_pretrained('./kcbert_finetuned')

# 모델을 평가 모드로 설정
model.eval()

# 질문과 문맥을 정의
question = "여주 공군사격장은 어디에 위치해 있나요?"
context = """미공군 매향리 사격장, 한ㆍ미합동훈련장, 여주 공군사격장, 수도권 비행금지구역, 휴전선 비행금지구역 이렇게 공역이 되어 있는데..."""

# 질문과 문맥을 토크나이즈
inputs = tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True)

# 모델 예측
with torch.no_grad():
    outputs = model(**inputs)

# 예측된 답변 위치 추출
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1

# 예측된 답변 디코딩
answer = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx])
print(f"질문: {question}")
print(f"답변: {answer}")


2. 모델 배포

In [ ]:
import streamlit as st
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch

# 학습된 KcBERT 모델과 토크나이저 로드
model_path = "./kcbert_finetuned"
tokenizer = BertTokenizerFast.from_pretrained(model_path)
model = BertForQuestionAnswering.from_pretrained(model_path)

# Streamlit 애플리케이션 구성
st.title("KcBERT 질문-답변 시스템")

context = st.text_area("문맥을 입력하세요", "여기에 문맥을 입력하세요.")
question = st.text_input("질문을 입력하세요")

if st.button("답변 생성"):
    if question and context:
        inputs = tokenizer(question, context, return_tensors="pt", max_length=512, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1
        answer = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx])
        st.write(f"답변: {answer}")
    else:
        st.write("질문과 문맥을 입력해주세요.")


3. 추가 학습 (Fine-tuning)
학습이 충분하지 않거나 더 많은 데이터를 사용해 성능을 향상시키고 싶다면, 추가로 학습을 진행할 수 있습니다. 이를 통해 모델의 성능을 더욱 개선할 수 있습니다.

4. 모델 평가 (Evaluation)
학습된 모델을 평가하기 위한 데이터셋을 준비하고, 정확도(accuracy), F1 점수 등 평가 지표를 통해 모델 성능을 검증할 수 있습니다.

In [ ]:
from datasets import load_metric

# 평가 지표 설정
metric = load_metric("accuracy")

# 평가 데이터셋에 대해 예측 수행
for example in eval_dataset:
    inputs = tokenizer(example["question"], example["context"], return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits) + 1
    predicted_answer = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx])

    # 평가 메트릭 계산
    metric.add(prediction=predicted_answer, reference=example["answer"])

# 최종 평가 결과 출력
print(metric.compute())


5. Hugging Face에 모델 업로드

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

# 모델 업로드
model.push_to_hub("your-model-name")
tokenizer.push_to_hub("your-model-name")
